In [1]:
import vastai
import asyncio
import random
import logging

logging.getLogger("Serverless").propagate = False
# or, if the library uses a package logger name:
logging.getLogger("vastai").propagate = False


In [ ]:
import asyncio
import random
import vastai

async def start_session():
    client = vastai.Serverless()
    session_a = None
    session_b = None

    try:
        endpoint = await client.get_endpoint("my-comfy-endpoint")

        payload = {
            "input": {
                "modifier": "Text2Image",
                "modifications": {
                    "prompt": "Generate a page from a peanuts comic strip.",
                    "width": 512,
                    "height": 512,
                    "steps": 10,
                    "seed": random.randint(1, 1000)
                }
            }
        }

        session_a = await endpoint.session()
        session_b = await endpoint.session()

        def print_response(response):
            print(response["response"]["output"][0]["local_path"])

        while True:
            await asyncio.gather(
                session_a.request("/generate/sync", payload).then(print_response),
                session_b.request("/generate/sync", payload).then(print_response),
            )

    except asyncio.CancelledError:
        print("Notebook cell cancelled.")
        raise

    finally:
        if session_a:
            await session_a.close()
        if session_b:
            await session_b.close()
        await client.close()
        print("Sessions closed.")


In [ ]:
await start_session()

In [ ]:
import asyncio
import random
import vastai
def status_printout(status):
    status_obj = status["response"]["status"]

    print(
        f"{status_obj['message']:<30} | "
        f"Train loss: {status_obj['train_loss']:>8.4f} | "
        f"Accuracy: {status_obj['val_acc'] * 100:>6.2f}%"
    )


async def start_training_session():
    async with vastai.Serverless() as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint-2")

        payload = {'max_train_batches_per_epoch': 100, "epochs" : 200}



        async with await endpoint.session() as session:
            async def on_close(session):
                await session.request(route="/cancel_task", payload={})
            session.set_on_close(on_close)
            await session.request(route="/start_task", payload=payload)
            count = 0
            while True:
                await asyncio.sleep(1)
                status = await session.request(route="/status", payload={})
                status_printout(status)
                if status["response"]["status"]["state"] != "running":
                    break
                if count > 15:
                    status = await session.request(route="/cancel_task", payload={})
                count += 1
        print("Training complete")

    print("Sessions closed.")


In [30]:
await asyncio.gather(start_training_session())

Training epoch 2/200 batch 40/100 | Train loss:   0.2025 | Accuracy:  90.97%
Training epoch 3/200 batch 100/100 | Train loss:   0.1240 | Accuracy:  95.16%
Training epoch 5/200 batch 40/100 | Train loss:   0.0951 | Accuracy:  96.56%
Training epoch 6/200 batch 100/100 | Train loss:   0.0836 | Accuracy:  97.03%
Training epoch 8/200 batch 50/100 | Train loss:   0.0810 | Accuracy:  97.12%
Training epoch 9/200 batch 100/100 | Train loss:   0.0614 | Accuracy:  97.62%
Training epoch 11/200 batch 60/100 | Train loss:   0.0569 | Accuracy:  97.62%
Validation epoch 12/200 complete | Train loss:   0.0451 | Accuracy:  97.59%
Training epoch 14/200 batch 80/100 | Train loss:   0.0406 | Accuracy:  97.81%
Training epoch 16/200 batch 10/100 | Train loss:   0.0596 | Accuracy:  97.81%
Training epoch 17/200 batch 100/100 | Train loss:   0.0465 | Accuracy:  98.00%
Training epoch 19/200 batch 40/100 | Train loss:   0.0386 | Accuracy:  98.47%
Training epoch 20/200 batch 100/100 | Train loss:   0.0296 | Accurac

[None]

In [31]:
await asyncio.gather(start_training_session(), start_training_session())

Training epoch 2/200 batch 40/100 | Train loss:   0.2025 | Accuracy:  90.97%
Training epoch 3/200 batch 100/100 | Train loss:   0.1239 | Accuracy:  95.12%
Training epoch 5/200 batch 60/100 | Train loss:   0.0940 | Accuracy:  96.62%
Training epoch 6/200 batch 100/100 | Train loss:   0.0830 | Accuracy:  97.03%
Training epoch 8/200 batch 90/100 | Train loss:   0.0770 | Accuracy:  97.12%
Training epoch 10/200 batch 20/100 | Train loss:   0.0502 | Accuracy:  97.72%
Training epoch 11/200 batch 100/100 | Train loss:   0.0556 | Accuracy:  97.72%
Training epoch 13/200 batch 70/100 | Train loss:   0.0465 | Accuracy:  97.47%
Training epoch 15/200 batch 10/100 | Train loss:   0.0418 | Accuracy:  97.56%
Training epoch 16/200 batch 100/100 | Train loss:   0.0507 | Accuracy:  97.75%
Training epoch 18/200 batch 30/100 | Train loss:   0.0462 | Accuracy:  97.84%
Training epoch 19/200 batch 100/100 | Train loss:   0.0372 | Accuracy:  98.53%
Training epoch 21/200 batch 50/100 | Train loss:   0.0316 | Accu

CancelledError: 

In [19]:
import asyncio
import random
import vastai
def status_printout(status):
    status_obj = status["response"]["status"]

    print(
        f"{status_obj['message']:<30} | "
        f"Train loss: {status_obj['train_loss']:>8.4f} | "
        f"Accuracy: {status_obj['val_acc'] * 100:>6.2f}%"
    )


async def start_full_training_session():
    async with vastai.Serverless() as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")





        session = await endpoint.session()
        payload = {'max_train_batches_per_epoch': 10, "epochs" : 20, "session_id": session.session_id}
        await session.request(route="/start_task", payload=payload)
        while True:
            await asyncio.sleep(1)
            status = await session.request(route="/status", payload={}, retry=False)
            print(status)
            #status_printout(status)
        print("Training complete")

    print("Sessions closed.")


In [20]:
await asyncio.gather(start_full_training_session())

{'response': {'ok': True, 'status': {'task_id': '5VhxuE6AOMZXE', 'state': 'running', 'message': 'Validation epoch 2/20 complete', 'created_at': 1766011801.3936226, 'started_at': 1766011801.393623, 'finished_at': None, 'epoch': 2, 'step': 20, 'total_steps': 200, 'train_loss': 1.234742820262909, 'train_acc': 0.6984375, 'val_loss': 0.8508174633979797, 'val_acc': 0.79, 'last_update_at': 1766011802.5518765, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': '5VhxuE6AOMZXE'}, 'error_type': None, 'error': None}}, 'latency': 0.18895339965820312, 'url': 'https://145.236.166.111:41798', 'reuqest_idx': 0, 'auth_data': {'__request_id': '24851e66-6b4a-45f9-8859-46cf7fc9ecd2', 'cost': 100.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 9, 'signature': 'mKo2taDu8NBrOgLLqOVljA8K8e+5FwUiA2ISc86dqZ53fWBbeYZIBj+fw8vrMjBpK43

CancelledError: 

In [1]:
import vastai
import asyncio

In [2]:
my_data = []
my_data_lock = asyncio.Lock()

In [1]:
my_data = [{'session_id': 'OJ2Oz7unDo3ed', 'session_auth': {'__request_id': 'b8bcc36d-b8b3-4d2d-a0fe-fa639f37000c', 'cost': 100.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 12, 'signature': 'hGhgJH1FRNUf8ZWypSM3abPkc89Ze86PYHp9d3U399I3irZ5tdGOigBAqT5ZrHj0ziBci57mv6+6NcwxD8qfsnOi8f6n2DKIbNSyx39hnvLWziiOoGH4BZhhhc3PLj2vzapgmbE4eQ5l5xWam0pphIj0cqZwZDGxk3dl3a59k/fzEL1A4Suc+ZvI1a8T+OMYTgKsZZQNNPOHz1IqZ4HeeSjowFj31hLA9cQPG2gp+i0/I51nNgc1tOvkivEUS6EemQy2DKOop5iY7iecjEjrCZmq+gJ6BkerENQoZc29bcfN/7j9q2CX0EAZPDGrzd12IxwAFbeb1i26qb31HaHgXg==', 'url': 'https://145.236.166.111:41101'}, 'session_url': 'https://145.236.166.111:41101'}]


In [ ]:



async def create_session(client):
    endpoint = await client.get_endpoint("my-pytorch-endpoint")
    session = await endpoint.session(cost=10.0)

    results = {
        "session_id": session.session_id,
        "session_auth": session.auth_data,
    }

    async with my_data_lock:
        my_data.extend([results])

    return results



NUM_SESSIONS = 9

async def main():
    async with vastai.Serverless(debug=False) as client:
        tasks = [create_session(client) for _ in range(NUM_SESSIONS)]
        results = await asyncio.gather(*tasks)




await main()


Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route
Did not get request_idx from initial route


In [4]:
async def main():
    async with vastai.Serverless(debug=True) as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")

        session_data = my_data[0]

        session = await endpoint.get_session(session_data["session_id"], session_auth=session_data["session_auth"])
        if session is None:
            print("session is closed")
            return
        payload = {'max_train_batches_per_epoch': 10, "epochs" : 20, "session_id": session.session_id}
        await session.request(route="/start_task", payload=payload)
        while await session.is_open():
            await asyncio.sleep(1)
            try:
                status = await session.request(route="/status", payload={}, retry=False)
            except:
                print('session was closed')
                break
            print(status)
await main()

[2025-12-17 18:35:51,078] Serverless - INFO - Started aiohttp ClientSession
[2025-12-17 18:35:51,375] Serverless - INFO - Loaded Vast.ai SSL certificate
[2025-12-17 18:35:51,670] Serverless - INFO - Found 2 endpoints
[2025-12-17 18:35:51,848] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:51,848] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:51,918] Serverless - INFO - Endpoint request task completed
[2025-12-17 18:35:52,979] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:52,979] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:53,046] Serverless - INFO - Endpoint request task completed


{'response': {'ok': True, 'status': {'task_id': 'J5iUIUNozbfoQ', 'state': 'running', 'message': 'Validation epoch 2/20 complete', 'created_at': 1766025351.8858962, 'started_at': 1766025351.8858967, 'finished_at': None, 'epoch': 2, 'step': 20, 'total_steps': 200, 'train_loss': 1.23474280834198, 'train_acc': 0.6984375, 'val_loss': 0.8508174741268157, 'val_acc': 0.79, 'last_update_at': 1766025352.9558358, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'J5iUIUNozbfoQ'}, 'error_type': None, 'error': None}}, 'latency': 0.06621670722961426, 'url': 'https://136.59.129.136:34969', 'reuqest_idx': 0, 'auth_data': {'__request_id': 'f28da04c-dbcf-4e9b-8279-f7f671dc5408', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 109, 'signature': 'Szq3GoPNdlXjBdZhFzlqCUtwUK/j4HlX++TJ6LUSDozaiQLXn4B2QMMCTHN2tA8i5NS

[2025-12-17 18:35:54,106] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:54,107] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:54,171] Serverless - INFO - Endpoint request task completed


{'response': {'ok': True, 'status': {'task_id': 'J5iUIUNozbfoQ', 'state': 'running', 'message': 'Training epoch 6/20 batch 10/10', 'created_at': 1766025351.8858962, 'started_at': 1766025351.8858967, 'finished_at': None, 'epoch': 6, 'step': 60, 'total_steps': 200, 'train_loss': 0.3942483693361282, 'train_acc': 0.8890625, 'val_loss': 0.44132314026355746, 'val_acc': 0.8684375, 'last_update_at': 1766025354.0985534, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'J5iUIUNozbfoQ'}, 'error_type': None, 'error': None}}, 'latency': 0.0638275146484375, 'url': 'https://136.59.129.136:34969', 'reuqest_idx': 0, 'auth_data': {'__request_id': 'f28da04c-dbcf-4e9b-8279-f7f671dc5408', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 109, 'signature': 'Szq3GoPNdlXjBdZhFzlqCUtwUK/j4HlX++TJ6LUSDozaiQLXn4B2QMMCTHN

[2025-12-17 18:35:55,231] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:55,232] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:55,295] Serverless - INFO - Endpoint request task completed


...

{'response': {'ok': True, 'status': {'task_id': 'J5iUIUNozbfoQ', 'state': 'running', 'message': 'Training epoch 15/20 batch 10/10', 'created_at': 1766025351.8858962, 'started_at': 1766025351.8858967, 'finished_at': None, 'epoch': 15, 'step': 150, 'total_steps': 200, 'train_loss': 0.17485131174325944, 'train_acc': 0.9484375, 'val_loss': 0.25134636610746386, 'val_acc': 0.925625, 'last_update_at': 1766025357.4609878, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'J5iUIUNozbfoQ'}, 'error_type': None, 'error': None}}, 'latency': 0.06371116638183594, 'url': 'https://136.59.129.136:34969', 'reuqest_idx': 0, 'auth_data': {'__request_id': 'f28da04c-dbcf-4e9b-8279-f7f671dc5408', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 109, 'signature': 'Szq3GoPNdlXjBdZhFzlqCUtwUK/j4HlX++TJ6LUSDozaiQLXn4B2QMM

[2025-12-17 18:35:58,606] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:58,607] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:58,684] Serverless - INFO - Endpoint request task completed


{'response': {'ok': True, 'status': {'task_id': 'J5iUIUNozbfoQ', 'state': 'running', 'message': 'Training epoch 18/20 batch 10/10', 'created_at': 1766025351.8858962, 'started_at': 1766025351.8858967, 'finished_at': None, 'epoch': 18, 'step': 180, 'total_steps': 200, 'train_loss': 0.17782035544514657, 'train_acc': 0.9515625, 'val_loss': 0.2290162818133831, 'val_acc': 0.9253125, 'last_update_at': 1766025358.5820968, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'J5iUIUNozbfoQ'}, 'error_type': None, 'error': None}}, 'latency': 0.07689785957336426, 'url': 'https://136.59.129.136:34969', 'reuqest_idx': 0, 'auth_data': {'__request_id': 'f28da04c-dbcf-4e9b-8279-f7f671dc5408', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 109, 'signature': 'Szq3GoPNdlXjBdZhFzlqCUtwUK/j4HlX++TJ6LUSDozaiQLXn4B2QMM

[2025-12-17 18:35:59,745] Serverless - INFO - Queued endpoint request
[2025-12-17 18:35:59,746] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 18:35:59,807] Serverless - ERROR - Attempt 1 failed: HTTP 410 from https://136.59.129.136:34969/status: {"error": "invalid session"}
[2025-12-17 18:35:59,808] Serverless - ERROR - Request errored: Too many retries for https://136.59.129.136:34969/status (last_status=410, last_text='{"error": "invalid session"}')
[2025-12-17 18:35:59,808] Serverless - INFO - Closed aiohttp ClientSession


session was closed


In [5]:
print(len(my_data))

0


In [4]:
async def run_session(endpoint, session_data):
    session = await endpoint.get_session(
        session_data["session_id"],
        session_auth=session_data["session_auth"],
    )

    if session is None:
        print(f"Session {session_data['session_id']} is closed")
        return

    payload = {
        "max_train_batches_per_epoch": 10,
        "epochs": 20,
        "session_id": session.session_id,
    }

    await session.request(route="/start_task", payload=payload)

    while await session.is_open():
        await asyncio.sleep(1)
        try:
            status = await session.request(
                route="/status",
                payload={},
                retry=False,
            )
            print(f"Session {session.session_id} status:", status)
        except Exception:
            print(f"Session {session.session_id} was closed")
            break


async def main():
    async with vastai.Serverless(debug=False) as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")

        tasks = [
            run_session(endpoint, session_data)
            for session_data in my_data
        ]

        await asyncio.gather(*tasks)


await main()


Session mk0ZQiQFxlmPw status: {'response': {'ok': True, 'status': {'task_id': 'mk0ZQiQFxlmPw', 'state': 'running', 'message': 'Training epoch 3/20 batch 10/10', 'created_at': 1766031023.8604953, 'started_at': 1766031023.8604958, 'finished_at': None, 'epoch': 3, 'step': 30, 'total_steps': 200, 'train_loss': 0.7177634537220001, 'train_acc': 0.7828125, 'val_loss': 0.8508174490928649, 'val_acc': 0.79, 'last_update_at': 1766031024.8140173, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'mk0ZQiQFxlmPw'}, 'error_type': None, 'error': None}}, 'latency': 0.025642871856689453, 'url': 'https://45.29.62.113:20163', 'request_idx': 0, 'auth_data': {'__request_id': '3b9f19e8-3b12-4f23-b46a-d82645278514', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 107, 'request_idx': 138, 'signature': 'M0JPEg8spFuTIV8tsUpSrv5o6PU8oZnK0

Attempt 1 failed: HTTP 410 from https://58.224.7.136:30566/status: {"error": "invalid session"}
Request errored: Too many retries for https://58.224.7.136:30566/status (last_status=410, last_text='{"error": "invalid session"}')


Session khVPvAwE4kAeR was closed
Session ALRX5IqHWHMSF status: {'response': {'ok': True, 'status': {'task_id': 'ALRX5IqHWHMSF', 'state': 'running', 'message': 'Training epoch 10/20 batch 10/10', 'created_at': 1766031024.3328023, 'started_at': 1766031024.3328028, 'finished_at': None, 'epoch': 10, 'step': 100, 'total_steps': 200, 'train_loss': 0.23956973403692244, 'train_acc': 0.9328125, 'val_loss': 0.3036499707400799, 'val_acc': 0.9075, 'last_update_at': 1766031028.1106496, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'ALRX5IqHWHMSF'}, 'error_type': None, 'error': None}}, 'latency': 0.1595292091369629, 'url': 'https://85.195.201.222:40167', 'request_idx': 0, 'auth_data': {'__request_id': '3b9f19e8-3b12-4f23-b46a-d82645278514', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 1, 'request_idx': 137, 'signature

Attempt 1 failed: HTTP 410 from https://87.205.21.33:42952/status: {"error": "invalid session"}
Request errored: Too many retries for https://87.205.21.33:42952/status (last_status=410, last_text='{"error": "invalid session"}')


Session ltkV64Lr155NS status: {'response': {'ok': True, 'status': {'task_id': 'ltkV64Lr155NS', 'state': 'running', 'message': 'Training epoch 19/20 batch 10/10', 'created_at': 1766031024.392542, 'started_at': 1766031024.3925424, 'finished_at': None, 'epoch': 19, 'step': 190, 'total_steps': 200, 'train_loss': 0.20069760382175444, 'train_acc': 0.9390625, 'val_loss': 0.1925952558964491, 'val_acc': 0.9475, 'last_update_at': 1766031028.4221177, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'ltkV64Lr155NS'}, 'error_type': None, 'error': None}}, 'latency': 0.17401742935180664, 'url': 'https://109.228.173.28:30362', 'request_idx': 0, 'auth_data': {'__request_id': 'df8931c1-adb1-428a-a091-80fbe860a777', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 133, 'signature': 'K8urDVZIyER7bMVbyO6x9kkKY2AdG

...

Session ALRX5IqHWHMSF status: {'response': {'ok': True, 'status': {'task_id': 'ALRX5IqHWHMSF', 'state': 'running', 'message': 'Validation epoch 20/20 complete', 'created_at': 1766031024.3328023, 'started_at': 1766031024.3328028, 'finished_at': None, 'epoch': 20, 'step': 200, 'total_steps': 200, 'train_loss': 0.14327742867171764, 'train_acc': 0.9640625, 'val_loss': 0.17572281196713446, 'val_acc': 0.9459375, 'last_update_at': 1766031032.1396298, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'ALRX5IqHWHMSF'}, 'error_type': None, 'error': None}}, 'latency': 0.15956568717956543, 'url': 'https://85.195.201.222:40167', 'request_idx': 0, 'auth_data': {'__request_id': '3b9f19e8-3b12-4f23-b46a-d82645278514', 'cost': 10.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 1, 'request_idx': 137, 'signature': 'XuEmu5x0KdzsA2/hi5ZFSc5QZ

Attempt 1 failed: HTTP 410 from https://74.48.140.178:27356/status: {"error": "invalid session"}
Request errored: Too many retries for https://74.48.140.178:27356/status (last_status=410, last_text='{"error": "invalid session"}')


Session srAEUAm1UFjk4 was closed


Attempt 1 failed: HTTP 410 from https://90.224.159.6:40969/status: {"error": "invalid session"}
Request errored: Too many retries for https://90.224.159.6:40969/status (last_status=410, last_text='{"error": "invalid session"}')


Session 0z6SlB08J3sjB was closed


In [4]:
import asyncio
import random
import vastai

async def main():
    async with vastai.Serverless(debug=True) as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")

        session_data = my_data[0]

        session = await endpoint.get_session(session_data["session_id"], session_auth=session_data["session_auth"])
        await session.close()
await main()

[2025-12-17 16:17:38,097] Serverless - INFO - Started aiohttp ClientSession
[2025-12-17 16:17:38,097] Serverless - INFO - Started aiohttp ClientSession
[2025-12-17 16:17:38,397] Serverless - INFO - Loaded Vast.ai SSL certificate
[2025-12-17 16:17:38,397] Serverless - INFO - Loaded Vast.ai SSL certificate
[2025-12-17 16:17:38,716] Serverless - INFO - Found 1 endpoints
[2025-12-17 16:17:38,716] Serverless - INFO - Found 1 endpoints
[2025-12-17 16:17:39,264] Serverless - INFO - Queued endpoint request
[2025-12-17 16:17:39,264] Serverless - INFO - Queued endpoint request
[2025-12-17 16:17:39,265] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 16:17:39,265] Serverless - DEBUG - Found worker machine, starting work
[2025-12-17 16:17:39,447] Serverless - INFO - Endpoint request task completed
[2025-12-17 16:17:39,447] Serverless - INFO - Endpoint request task completed
[2025-12-17 16:17:39,448] Serverless - INFO - Closed aiohttp ClientSession
[2025-12-17 16:17:39,448] Ser